In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
df = pd.read_csv("data.csv", sep=";", decimal=",")
df = df.rename(columns={
    "Column1": "Date",
    "Column2": "SPX",
    "Column3": "S5SFTW",
    "Column4": "S5PHRM",
    "Column5": "S5CPGS",
    "Column6": "S5ENRSX",
    "Column7": "S5FDBT",
    "Column8": "S5TECH",
    "Column9": "S5RETL",
    "Column10": "S5BANKX",
    "Column11": "S5HCES",
    "Column12": "S5DIVF",
    "Column13": "S5UTILX",
    "Column14": "S5MEDA",
    "Column15": "S5REAL",
    "Column16": "S5TELSX",
    "Column17": "S5MATRX",
    "Column18": "S5INSU",
    "Column19": "S5FDSR",
    "Column20": "S5HOUS",
    "Column21": "S5SSEQX",
    "Column22": "S5TRAN",
    "Column23": "S5HOTR",
    "Column24": "S5CODU",
    "Column25": "S5AUCO",
    "Column26": "S5COMS",
})
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
df["MonthIndex"] = df["Date"].dt.to_period("M")

#----------
#PARAMETERS
#----------
fees_rate=0.0002
# const
df_length = df.shape[1] - 2  # bcs of date and spx
last_rebalance = df.loc[0, "Date"]  # première date
month_count = 0

#Return a percentage between a and b
def evolution(v1, v2):
    return round((v2 - v1) / v1 * 100, 5)

#-------------------
#SORTER BETWEEN PERF
#-------------------

def DecilSorter(start, hist, proportion=3):
    bestperformer = []
    worstperformer = []
    performerc = []

    for i in range(2, df_length-1):
      performerc.append((evolution(float(df.iloc[start, i]), float(df.iloc[start + hist, i])), i))

    performerc.sort(reverse=True)
    offset=proportion
    for i in range(proportion):
        bestperformer.append(performerc[i][1])

    for i in range(len(performerc) - offset - proportion, len(performerc) - offset):
        worstperformer.append(performerc[i][1])


    return bestperformer, worstperformer


#--------------
#ALGO TITMANJEG
#--------------

def JeegadesshTitmanStrat(start=0, hold=6, hist=6, proportion=3):
    #new dataframe for stock quantity
    Indexcolumns = ["Date","SPX","S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS","Money"]
    Tickercolumns=["S5SFTW","S5PHRM","S5CPGS","S5ENRSX","S5FDBT","S5TECH","S5RETL","S5BANKX","S5HCES","S5DIVF","S5UTILX","S5MEDA","S5REAL","S5TELSX","S5MATRX","S5INSU","S5FDSR","S5HOUS","S5SSEQX","S5TRAN","S5HOTR","S5CODU","S5AUCO","S5COMS"]
    StockQty = df.copy()
    StockQty.drop(columns="MonthIndex", inplace=True)
    StockQty.loc[:, :] = 0

    #starting data
    MoneyAtStart = 10000000
    month_count=0
    CurrentValue=MoneyAtStart

    #first ligne
    StockQty.loc[0, "Money"] = MoneyAtStart
    StockQty.loc[0, "SPX"] = df.iloc[0, 1]
    StockQty.loc[0, "Date"] = df.iloc[0, 0]

    #start of the algorithm

    for i in range(1,df.shape[0]):
      StockQty.iloc[i,0]=df.iloc[i,0]
      StockQty.iloc[i,1]=df.iloc[i,1]
      fees=0

      if df.loc[i, "Date"].month != df.loc[i-1, "Date"].month:
        month_count += 1


    # Si on atteint la période voulue
      if (i>= hist and month_count % hold == 0 and df.loc[i, "Date"].month != df.loc[i - 1, "Date"].month):
        #print(f"🔁 Rebalancement déclenché à la date : {df.loc[i, 'Date'].date()}")
        bestperformer, worstperformer = DecilSorter(i-hist, hist, proportion)


        for bestperf in bestperformer:
          StockQty.iloc[i,bestperf]=round((CurrentValue/(len(bestperformer)*2))/float(df.iloc[i,bestperf]),2)


        for worstperf in worstperformer:
          StockQty.iloc[i,worstperf]=round(-1*(CurrentValue/(len(worstperformer)*2))/float(df.iloc[i,worstperf]),2)

        for stocks in range(2,StockQty.shape[1]-1):
          if StockQty.iloc[i,stocks]!=StockQty.iloc[i-1,stocks]:
            fees+=abs(StockQty.iloc[i,stocks]-StockQty.iloc[i-1,stocks])*df.iloc[i,stocks]*fees_rate

      else :
        for stocks in range(2,StockQty.shape[1]-1):
          StockQty.iloc[i,stocks]=StockQty.iloc[i-1,stocks] #same qty
      #value of pf

      GainOrLoss = 0
      for stocks in range(2, StockQty.shape[1]-1):
          qty = StockQty.iloc[i, stocks]
          if qty != 0.0:
            price_now = df.iloc[i, stocks]
            price_prev = df.iloc[i - 1, stocks]
            GainOrLoss += qty * (price_now - price_prev)


      CurrentValue+=GainOrLoss-fees
      StockQty.iloc[i,-1]=CurrentValue

    return StockQty

results = JeegadesshTitmanStrat(0, 3, 9*22 , 3)


#---- GRAPH

"""
money_norm =(results["Money"]/10000000*100)-100
spx_norm=(results["SPX"]/results["SPX"].iloc[0]* 100)-100

fix = px.line(x=results["Date"], y=[money_norm, spx_norm],labels={"value":"Évolution en %", "variable":"Série"},title="Comparaison des évolutions en %")
fix.update_layout(hovermode="x unified")

fix.show()
"""

#-------- ANNUALISATION

AnnualizedDf=results[["Date","SPX","Money"]]
AnnualizedDf['Date'] = pd.to_datetime(AnnualizedDf['Date'])
AnnualizedDf['Year'] = AnnualizedDf['Date'].dt.year



YearList=AnnualizedDf["Year"].unique()
SPXAnnualized=pd.DataFrame(columns=YearList)
StratAnnualized=pd.DataFrame(columns=YearList)



for year in YearList:
  compteurPerYear=0
  for i in AnnualizedDf.index:
    if AnnualizedDf.loc[i,"Year"]==year:
      if compteurPerYear==0:
        SPXAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"SPX"]
        StratAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"Money"]
      else :
        SPXAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"SPX"]/SPXAnnualized.loc[0,year]*100-100
        StratAnnualized.loc[compteurPerYear,year]=AnnualizedDf.loc[i,"Money"]/StratAnnualized.loc[0,year]*100-100
      compteurPerYear+=1

for year in YearList:
  SPXAnnualized.loc[0,year]=SPXAnnualized.loc[0,year]/SPXAnnualized.loc[0,year]*100-100
  StratAnnualized.loc[0,year]=StratAnnualized.loc[0,year]/StratAnnualized.loc[0,year]*100-100



SPXAvg=[]
StratAvg=[]
for i in SPXAnnualized.index:
  sumSPX=0
  sumStrat=0
  for year in SPXAnnualized.columns:
    sumSPX+=SPXAnnualized.loc[i,year]
    sumStrat+=StratAnnualized.loc[i,year]
  SPXAvg.append(sumSPX/len(YearList))
  StratAvg.append(sumStrat/len(YearList))

#------- GRAPH / YEAR
"""
fig = px.line(SPXAnnualized, x=SPXAnnualized.index, y=SPXAnnualized.columns,labels={"value":"Returns in % of the SPX", "variable":"Years"},title="Comparaison of the returns on differents years | SPX")
fig.update_xaxes(tickmode='linear',  dtick=1,)
fig.show()


fig = px.line(StratAnnualized, x=StratAnnualized.index, y=StratAnnualized.columns,labels={"value":"Returns in % of the Start", "variable":"Years"},title="Comparaison of the returns on differents years | Strat")
fig.update_xaxes(tickmode='linear',  dtick=1,)
fig.show()
"""

SPXAnnualized=SPXAnnualized.drop(columns=2024) #too much nan
StratAnnualized=StratAnnualized.drop(columns=2024)

SPXAvg=[]
StratAVG=[]

for i in SPXAnnualized.index:
  sumSPX=0
  sumStrat=0
  for year in SPXAnnualized.columns:
    sumSPX+=SPXAnnualized.loc[i,year]
    sumStrat+=StratAnnualized.loc[i,year]
  SPXAvg.append(sumSPX/len(YearList))
  StratAVG.append(sumStrat/len(YearList))

dff = pd.DataFrame({"Index": (range(len(SPXAvg))),"SPX": SPXAvg,"Strat": StratAVG})


fig = px.line(dff, x="Index", y=["SPX","Strat"])
fig.show()


#---------- VOL


VolSPX = df[["Date", "SPX"]].copy()
VolSPX["Date"] = pd.to_datetime(VolSPX["Date"], dayfirst=True)
VolSPX["Year"] = VolSPX["Date"].dt.year
VolSPX["Return"] = np.log(VolSPX["SPX"] / VolSPX["SPX"].shift(1))
NumberofDayInYear=VolSPX.groupby("Year")["SPX"].count()
VolSPX = (VolSPX.groupby("Year")["Return"].std())
VolSPX = VolSPX * np.sqrt(NumberofDayInYear)


VolStrat=AnnualizedDf[["Money","Year"]].copy()
VolStrat["Money"]=VolStrat["Money"].astype(float)
VolStrat["Return"] = np.log(VolStrat["Money"] / VolStrat["Money"].shift(1))
NumberofDayInStrat=VolStrat.groupby("Year")["Money"].count()
VolStrat = (VolStrat.groupby("Year")["Return"].std())
VolStrat = VolStrat * np.sqrt(NumberofDayInStrat)

Vol=pd.concat([VolSPX,VolStrat],axis=1)
Vol.columns=["SPX","Strat"]


fig = px.line(Vol, x=Vol.index, y=Vol.columns,labels={"value":"Vol",},title="Comparaison of vol : SPX / Strat")
fig.update_xaxes(tickmode='linear',  dtick=1,)
fig.show()


C:\Users\Malob\AppData\Local\Temp\ipykernel_20228\3385403260.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  StockQty.loc[:, :] = 0
C:\Users\Malob\AppData\Local\Temp\ipykernel_20228\3385403260.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2002-01-01 00:00:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  StockQty.loc[0, "Date"] = df.iloc[0, 0]
C:\Users\Malob\AppData\Local\Temp\ipykernel_20228\3385403260.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [3]:
results

,Date,SPX,S5SFTW,S5PHRM,S5CPGS,S5ENRSX,S5FDBT,S5TECH,S5RETL,S5BANKX,...,S5INSU,S5FDSR,S5HOUS,S5SSEQX,S5TRAN,S5HOTR,S5CODU,S5AUCO,S5COMS,Money
0,2002-01-01 00:00:00,1148.08,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
1,2002-01-02 00:00:00,1154.67,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
2,2002-01-03 00:00:00,1165.27,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
3,2002-01-04 00:00:00,1172.51,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
4,2002-01-07 00:00:00,1164.89,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.000000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,2024-02-23 00:00:00,5088.80,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.466253e+07
5779,2024-02-26 00:00:00,5069.53,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.466167e+07
5780,2024-02-27 00:00:00,5078.18,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.467943e+07
5781,2024-02-28 00:00:00,5069.76,0.0,-1809.24,0.0,-3606.44,0.0,0.0,593.58,0.0,...,0.0,0.0,0.0,703.86,0.0,0.0,0.0,0.0,0.0,1.462293e+07


In [4]:
#the idea is the following

#create firstly the P matrix having wi,t= (-1/3 ; 0 ; 1/3) with i e [1,24] and t up to today, depending on the sign of the quantity in the result
c=0
PMatrix=np.zeros((0,26))
#generate returns clumns based on Money columns on df result
meanReturns=[]
results["Returns"] = results["Money"].pct_change()
for lines in range(1,len(results)):
    positionList=[]
    positionList.append(results.iloc[lines,0])
    meanReturns.append(results.iloc[lines,-1])
    for j in range(2,26):
        if results.iloc[lines-1,j]!=results.iloc[lines,j] and results.iloc[lines,j]!=0.0 :
            if results.iloc[lines,j]>0:
                positionList.append(1/3)
            else :
                positionList.append(-1/3)
        else :
            positionList.append(0)

    abssum=0
    for value in range(1,len(positionList)-1):
        abssum=abssum+abs(positionList[value])
    if abssum!=0 :
        positionList.append(np.mean(meanReturns)*92)
        PMatrix=np.vstack((PMatrix,positionList))
        meanReturns=[]


PMatrix

array([[Timestamp('2003-01-01 00:00:00'), 0, 0, ..., 0, 0,
        np.float64(-7.049809512746714e-05)],
       [Timestamp('2003-04-01 00:00:00'), 0, 0.3333333333333333, ..., 0,
        0, np.float64(0.02988303334002964)],
       [Timestamp('2003-07-01 00:00:00'), 0, 0, ..., 0, 0,
        np.float64(-0.054128591095722454)],
       ...,
       [Timestamp('2023-07-03 00:00:00'), 0.3333333333333333,
        -0.3333333333333333, ..., 0, 0, np.float64(0.07481522814748362)],
       [Timestamp('2023-10-02 00:00:00'), 0, 0, ..., 0, 0,
        np.float64(0.04689981426583853)],
       [Timestamp('2024-01-01 00:00:00'), 0, -0.3333333333333333, ..., 0,
        0, np.float64(0.029950607269093904)]],
      shape=(85, 26), dtype=object)

In [5]:
Q=PMatrix[:,-1]
P=PMatrix[:,1:-1]
D=PMatrix[:,1]

In [10]:
def getCorrelationMatrix(date,lookBackInDays,df):
    #check if date-lookback in df
    #extract all the lines for a lookback period since date using df
    end_date = pd.to_datetime(date)
    start_date = end_date - pd.Timedelta(days=lookBackInDays)
    print(list(df["Date"]))
    print(start_date)
    if start_date not in df["Date"]:
        raise ValueError("Start date not in dataframe")
    if end_date not in df["Date"]:
        raise ValueError("End date not in dataframe")

    ShortDf=df.copy(df[(df["Date"] >= start_date) & (df["Date"] <= end_date)])
    ShortDf=ShortDf.reset_index(drop=True)
    return ShortDf

ShortDf=getCorrelationMatrix("2022-03-01",180, results)

[Timestamp('2002-01-01 00:00:00'), Timestamp('2002-01-02 00:00:00'), Timestamp('2002-01-03 00:00:00'), Timestamp('2002-01-04 00:00:00'), Timestamp('2002-01-07 00:00:00'), Timestamp('2002-01-08 00:00:00'), Timestamp('2002-01-09 00:00:00'), Timestamp('2002-01-10 00:00:00'), Timestamp('2002-01-11 00:00:00'), Timestamp('2002-01-14 00:00:00'), Timestamp('2002-01-15 00:00:00'), Timestamp('2002-01-16 00:00:00'), Timestamp('2002-01-17 00:00:00'), Timestamp('2002-01-18 00:00:00'), Timestamp('2002-01-21 00:00:00'), Timestamp('2002-01-22 00:00:00'), Timestamp('2002-01-23 00:00:00'), Timestamp('2002-01-24 00:00:00'), Timestamp('2002-01-25 00:00:00'), Timestamp('2002-01-28 00:00:00'), Timestamp('2002-01-29 00:00:00'), Timestamp('2002-01-30 00:00:00'), Timestamp('2002-01-31 00:00:00'), Timestamp('2002-02-01 00:00:00'), Timestamp('2002-02-04 00:00:00'), Timestamp('2002-02-05 00:00:00'), Timestamp('2002-02-06 00:00:00'), Timestamp('2002-02-07 00:00:00'), Timestamp('2002-02-08 00:00:00'), Timestamp('20

ValueError: Start date not in dataframe

In [22]:
#---------
#PARAMETERS
#---------

free_asset=0 #proportion of risk free asset allocated in the benchmark
rf=0.046 #risk free rate
taux=0.01
RiskAversion=3.3

#kindly import it or calculate it !
Sigma = np.array([
    [0.0254, 0.0327, 0.0292, 0.0168],
    [0.0327, 0.0718, 0.0451, 0.0292],
    [0.0292, 0.0451, 0.0758, 0.0285],
    [0.0168, 0.0292, 0.0285, 0.0409]
], dtype=float)



#---------
#VIEWS
#---------


#Q :
confidence=2 #95
ViewMatrix=np.array([0.13,0.18,0.16,0.13]) #in %
ConfidenceMatrix=np.array([0.05,0.06,0.04,0.05])
OmegaMatrix=np.zeros((len(ViewMatrix),len(ViewMatrix)))

#𝛀
for i in range(OmegaMatrix.shape[0]):
    for j in range(OmegaMatrix.shape[1]):
      if i==j:
        OmegaMatrix[i][j]=(ConfidenceMatrix[i]/(confidence))**2

#𝛀-1
OmegaMatrixInverse = np.linalg.inv(OmegaMatrix)

#P
PMatrix=np.zeros((len(ViewMatrix),len(ViewMatrix)))
for i in range(PMatrix.shape[0]):
  PMatrix[i][i]=1


#---------
#BENCHMARK
#---------

def EW(ReturnofBenchmark,free_asset=0):
  tempoarray=np.zeros(len(µLT))
  for i in range(len(µLT)):
    tempoarray[i]=(1-free_asset)/len(µLT)
  return tempoarray


µLT=np.array([0.084,0.065,0.014,0.055])
weightBEW=EW(µLT)#possible to add free_asset liquity

#Perf of Benchmark
PerfOfBench=np.transpose(weightBEW)@µLT+free_asset*rf-rf
VarianceOfBench=np.transpose(weightBEW)@Sigma@weightBEW
lambdaOfBench=PerfOfBench/VarianceOfBench

µimplied=lambdaOfBench*Sigma@weightBEW+rf

#--------------
#MASTER FORMULA
#--------------


OptimisedReturn=(np.linalg.inv(np.linalg.inv(taux*Sigma)+np.transpose(PMatrix)@OmegaMatrixInverse@PMatrix))@(np.linalg.inv(taux*Sigma)@µimplied+np.transpose(PMatrix)@OmegaMatrixInverse@ViewMatrix)
print(OptimisedReturn)
print(" ")
#--------------------
#MARKOVITZ ALLOCATION
#--------------------

WeightBL=np.linalg.inv(Sigma)@(OptimisedReturn-rf)/RiskAversion
print(WeightBL)


RfWeightBL = 1.0 - float(WeightBL.sum())





[0.10037936 0.13897629 0.14011467 0.10475908]
 
[0.16147451 0.15598627 0.16850573 0.14023953]
